## Install Redis Client
- https://pypi.org/project/redis/

In [1]:
!pip3 install redis

     |████████████████████████████████| 173 kB 5.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


## Redis docker 
```bash
docker run --name redis -e REDIS_PORT_NUMBER=7000 -e ALLOW_EMPTY_PASSWORD=yes -p 7000:7000 bitnami/redis:latest
```

In [6]:
import redis
r = redis.Redis(host='127.0.0.1', port=7000, db=0)

In [7]:
r.set('foo', 'bar')

True

In [8]:
r.get('foo')

b'bar'

## Chapter 3

In [9]:
import time 
import threading

def notrans():
    print(r.incr('notrans:'))
    time.sleep(0.1)
    r.incr('notrans:', -1)
    
if 1:
    for i in range(3):
        threading.Thread(target=notrans).start()
    time.sleep(0.5)

1
23



In [10]:
def trans():
    pipeline = r.pipeline()
    pipeline.incr('trans:')
    time.sleep(0.1)
    pipeline.incr('trans:', -1)
    print(pipeline.execute()[0])
    
if 1:
    for i in range(3):
        threading.Thread(target=trans).start()
    time.sleep(0.5)

111




In [11]:
r.set('key', 'value')

True

In [12]:
r.get('key')

b'value'

In [13]:
r.expire('key', 2)

True

In [14]:
r.get('key')

In [15]:
r.set('key', 'value2')

True

In [16]:
r.expire('key', 100)

True

In [17]:
r.ttl('key')

97

In [18]:
r.ttl('key')

92

## Chapter 4
- persistence
    - snapshot
    - AOF (Append-only file)

In [19]:
def list_item(conn, itemid, sellerid, price):
    inventory = "inventory:%s" % sellerid
    item = "%s.%s" % (itemid, sellerid)
    end = time.time() + 5
    pipe = conn.pipeline()
    
    while time.time() < end:
        try:
            pipe.watch(inventory)
            if not pipe.sismember(inventory, itemid):
                pipe.unwatch()
                return None
            
            pipe.multi()
            pipe.zadd("market:", item, price)
            pipe.srem(inventory, itemid)
            pipe.execute()
            return True
        except redis.exceptions.WatchError:
            pass
    return False


In [20]:
# 3 ~ 5 round trips
def update_token(conn, token, user, item = None):
    timestamp = time.time()
    conn.hset('login:', token, user)
    conn.zadd('recent:', token, timestamp)
    if item:
        conn.zadd('viewed:' + token, item, timestamp)
        conn.zremrangebyrank('viewed:' + token, 0, -26)
        conn.zincrby('viewed:', item, -1)

In [21]:
def update_token_pipeline(conn, token, user, item = None):
    timestamp = time.time()
    pipe = conn.pipeline(False)
    pipe.hset('login:', token, user)
    pipe.zadd('recent:', token, timestamp)
    if item:
        pipe.zadd('viewed:' + token, item, timestamp)
        pipe.zremrangebyrank('viewed:' + token, 0, -26)
        pipe.zincrby('viewed:', item, -1)
    pipe.execute()

## Chapter 6